# Clasificación de Muestras Geológicas con Deep Learning (Google Colab)

Dataset: 5 clases (calcite, pyrite, quartz, Rocks, superficies_texturizadas)

Total: ~19,800 imágenes procedentes de tres fuentes:
- Minerales: calcite, pyrite, quartz (clases independientes)
- Rocas: agrupadas en una sola clase (Rocks)
- Superficies texturizadas: cracked, porous, wrinkled (agrupadas en una clase)

Modelos: CustomCNN (baseline) + ResNet18 (transfer learning)

**NOTA**: Asegúrate de activar GPU en Runtime > Change runtime type > Hardware accelerator > GPU

In [ ]:
# Verificar GPU disponible
!nvidia-smi

In [ ]:
# Clonar repositorio (opcional, si usas GitHub)
# !git clone https://github.com/tu-usuario/tu-repo.git
# %cd tu-repo

# O montar Google Drive (si tienes el proyecto ahí)
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/AF

In [ ]:
# Configuración inicial
import sys
sys.path.append('/content')

import torch
import matplotlib.pyplot as plt
import random

from src.dataset import get_dataloaders
from src.models import CustomCNN, get_resnet18
from src.train import train_model
from src.evaluate import evaluate_model, plot_confusion_matrix, plot_training_history
from src.utils import predict_from_dataset, visualize_prediction_from_dataset, visualize_dataset_samples

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {DEVICE}")
if DEVICE.type == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 1. Carga del Dataset

In [ ]:
# Carga de datos geológicos
DATA_DIR = 'data'
BATCH_SIZE = 32

train_loader, val_loader, test_loader, class_names = get_dataloaders(
    DATA_DIR,
    batch_size=BATCH_SIZE,
    val_split=0.15,
    test_split=0.15
)

print(f"Clases: {len(class_names)}")
print(f"Training samples: {len(train_loader.dataset)}")
print(f"Validation samples: {len(val_loader.dataset)}")
print(f"Test samples: {len(test_loader.dataset)}")

In [ ]:
# Clases del dataset
print("Clases geológicas:")
for i, class_name in enumerate(class_names, 1):
    print(f"{i}. {class_name}")

## 2. Visualización del Dataset

In [ ]:
# Muestra aleatoria de imágenes del dataset
visualize_dataset_samples(train_loader.dataset.dataset, class_names, n_samples=16)

## 3. Entrenamiento Custom CNN

Arquitectura: 4 bloques convolucionales (32→64→128→256)

Dataset: ~19,800 imágenes, 5 clases geológicas

In [ ]:
# Configuraciones para CustomCNN
cnn_configs = [
    {'name': 'Balanceada', 'epochs': 25, 'lr': 0.001, 'weight_decay': 1e-4},
    {'name': 'Conservadora', 'epochs': 35, 'lr': 0.0005, 'weight_decay': 1e-4},
    {'name': 'Muy Fina', 'epochs': 50, 'lr': 0.0001, 'weight_decay': 1e-3}
]

cnn_results = []

print("="*70)
print("CUSTOM CNN - PRUEBA DE CONFIGURACIONES")
print("="*70)

for i, config in enumerate(cnn_configs, 1):
    print(f"\n{'='*70}")
    print(f"CONFIG {i}/3: {config['name']}")
    print(f"Epochs: {config['epochs']} | LR: {config['lr']} | Weight Decay: {config['weight_decay']}")
    print(f"{'='*70}")
    
    model = CustomCNN(num_classes=len(class_names))
    
    history = train_model(
        model,
        train_loader,
        val_loader,
        epochs=config['epochs'],
        lr=config['lr'],
        weight_decay=config['weight_decay'],
        device=DEVICE
    )
    
    cnn_results.append({
        'config': config,
        'model': model,
        'history': history,
        'final_val_acc': history['val_acc'][-1],
        'final_val_loss': history['val_loss'][-1],
        'overfitting': history['train_acc'][-1] - history['val_acc'][-1]
    })
    
    print(f"\nResultados {config['name']}:")
    print(f"  Val Accuracy: {history['val_acc'][-1]:.2f}%")
    print(f"  Val Loss: {history['val_loss'][-1]:.4f}")
    print(f"  Overfitting: {history['train_acc'][-1] - history['val_acc'][-1]:.2f}%")

In [ ]:
# Comparación visual de configuraciones CustomCNN
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Custom CNN - Comparación de Configuraciones', fontsize=16, fontweight='bold')

for i, result in enumerate(cnn_results):
    ax = axes[i]
    history = result['history']
    config = result['config']
    
    ax.plot(history['train_acc'], label='Train Accuracy', linewidth=2)
    ax.plot(history['val_acc'], label='Val Accuracy', linewidth=2)
    ax.set_title(f"{config['name']}\n(epochs={config['epochs']}, lr={config['lr']})")
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Accuracy (%)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
plt.tight_layout()
plt.show()

In [ ]:
# Selección de mejor configuración CustomCNN
# Criterio: Mayor val_accuracy - penalización por overfitting
best_cnn = max(cnn_results, key=lambda x: x['final_val_acc'] - 0.3 * x['overfitting'])

print("="*70)
print("MEJOR CONFIGURACIÓN - CUSTOM CNN")
print("="*70)
print(f"Configuración: {best_cnn['config']['name']}")
print(f"  Epochs: {best_cnn['config']['epochs']}")
print(f"  Learning Rate: {best_cnn['config']['lr']}")
print(f"  Val Accuracy: {best_cnn['final_val_acc']:.2f}%")
print(f"  Val Loss: {best_cnn['final_val_loss']:.4f}")
print(f"  Overfitting: {best_cnn['overfitting']:.2f}%")
print("="*70)

## 4. Entrenamiento ResNet18 (Transfer Learning)

Transfer Learning desde ImageNet

Frozen: conv1, bn1, layer1, layer2

Trainable: layer3, layer4, fc

In [ ]:
# Configuraciones para ResNet18
resnet_configs = [
    {'name': 'Rápida', 'epochs': 12, 'lr': 0.001, 'weight_decay': 1e-4},
    {'name': 'Balanceada', 'epochs': 18, 'lr': 0.0005, 'weight_decay': 1e-4},
    {'name': 'Fina', 'epochs': 25, 'lr': 0.0001, 'weight_decay': 1e-4}
]

resnet_results = []

print("="*70)
print("RESNET18 (TRANSFER LEARNING) - PRUEBA DE CONFIGURACIONES")
print("="*70)

for i, config in enumerate(resnet_configs, 1):
    print(f"\n{'='*70}")
    print(f"CONFIG {i}/3: {config['name']}")
    print(f"Epochs: {config['epochs']} | LR: {config['lr']} | Weight Decay: {config['weight_decay']}")
    print(f"{'='*70}")
    
    model = get_resnet18(num_classes=len(class_names), pretrained=True, freeze_layers=True)
    
    history = train_model(
        model,
        train_loader,
        val_loader,
        epochs=config['epochs'],
        lr=config['lr'],
        weight_decay=config['weight_decay'],
        device=DEVICE
    )
    
    resnet_results.append({
        'config': config,
        'model': model,
        'history': history,
        'final_val_acc': history['val_acc'][-1],
        'final_val_loss': history['val_loss'][-1],
        'overfitting': history['train_acc'][-1] - history['val_acc'][-1]
    })
    
    print(f"\nResultados {config['name']}:")
    print(f"  Val Accuracy: {history['val_acc'][-1]:.2f}%")
    print(f"  Val Loss: {history['val_loss'][-1]:.4f}")
    print(f"  Overfitting: {history['train_acc'][-1] - history['val_acc'][-1]:.2f}%")

In [ ]:
# Comparación visual de configuraciones ResNet18
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('ResNet18 (Transfer Learning) - Comparación de Configuraciones', fontsize=16, fontweight='bold')

for i, result in enumerate(resnet_results):
    ax = axes[i]
    history = result['history']
    config = result['config']
    
    ax.plot(history['train_acc'], label='Train Accuracy', linewidth=2)
    ax.plot(history['val_acc'], label='Val Accuracy', linewidth=2)
    ax.set_title(f"{config['name']}\n(epochs={config['epochs']}, lr={config['lr']})")
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Accuracy (%)')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
plt.tight_layout()
plt.show()

In [ ]:
# Selección de mejor configuración ResNet18
best_resnet = max(resnet_results, key=lambda x: x['final_val_acc'] - 0.3 * x['overfitting'])

print("="*70)
print("MEJOR CONFIGURACIÓN - RESNET18")
print("="*70)
print(f"Configuración: {best_resnet['config']['name']}")
print(f"  Epochs: {best_resnet['config']['epochs']}")
print(f"  Learning Rate: {best_resnet['config']['lr']}")
print(f"  Val Accuracy: {best_resnet['final_val_acc']:.2f}%")
print(f"  Val Loss: {best_resnet['final_val_loss']:.4f}")
print(f"  Overfitting: {best_resnet['overfitting']:.2f}%")
print("="*70)

## 5. Evaluación en Test Set

In [ ]:
# Evaluación Custom CNN en test set
print("="*70)
print("EVALUACIÓN: MEJOR CUSTOM CNN")
print("="*70)
results_cnn = evaluate_model(best_cnn['model'], test_loader, class_names, device=DEVICE)

In [ ]:
# Evaluación ResNet18 en test set
print("\n" + "="*70)
print("EVALUACIÓN: MEJOR RESNET18")
print("="*70)
results_resnet = evaluate_model(best_resnet['model'], test_loader, class_names, device=DEVICE)

## 6. Comparación Final y Selección del Mejor Modelo

In [ ]:
# Comparación final en test set
print("\n" + "="*70)
print("COMPARACIÓN FINAL - TEST SET")
print("="*70)
print(f"Custom CNN ({best_cnn['config']['name']}): {results_cnn['accuracy']:.2f}%")
print(f"ResNet18 ({best_resnet['config']['name']}): {results_resnet['accuracy']:.2f}%")
print(f"\nMejora con Transfer Learning: {results_resnet['accuracy'] - results_cnn['accuracy']:.2f}%")
print("="*70)

# Selección del modelo ganador
if results_resnet['accuracy'] > results_cnn['accuracy']:
    winner_name = f"ResNet18 ({best_resnet['config']['name']})"
    winner_model = best_resnet['model']
    winner_results = results_resnet
else:
    winner_name = f"Custom CNN ({best_cnn['config']['name']})"
    winner_model = best_cnn['model']
    winner_results = results_cnn

print(f"\nMODELO GANADOR: {winner_name}")
print(f"Test Accuracy: {winner_results['accuracy']:.2f}%")

## 7. Matriz de Confusión

In [ ]:
# Matriz de confusión del modelo ganador
plot_confusion_matrix(
    winner_results['labels'],
    winner_results['predictions'],
    class_names,
    figsize=(10, 8)
)

## 8. Predicción en Imagen del Test Set

In [ ]:
# Predicción en imagen aleatoria del test set
test_dataset = test_loader.dataset.dataset
random_idx = random.randint(0, len(test_dataset) - 1)

predictions, true_label, image = predict_from_dataset(
    test_dataset,
    winner_model,
    class_names,
    random_idx,
    device=DEVICE,
    top_k=5
)

print(f"True Label: {true_label}")
print("\nTop 5 predictions:")
for i, (class_name, prob) in enumerate(predictions, 1):
    print(f"{i}. {class_name}: {prob:.2f}%")

In [ ]:
# Visualización de la predicción
visualize_prediction_from_dataset(image, predictions[:3], true_label)

## 9. Guardar Mejor Modelo

In [ ]:
# Guardar modelo ganador
import os
os.makedirs('models', exist_ok=True)

MODEL_PATH = 'models/best_model.pth'
torch.save(winner_model.state_dict(), MODEL_PATH)

print("="*70)
print("MODELO GUARDADO")
print("="*70)
print(f"Modelo: {winner_name}")
print(f"Path: {MODEL_PATH}")
print(f"Test Accuracy: {winner_results['accuracy']:.2f}%")
print("="*70)

In [ ]:
# Descargar modelo (opcional)
from google.colab import files
files.download(MODEL_PATH)